In [1]:
import pandas as pd
import sys
import os

# Add the scripts directory to the Python path
sys.path.append(os.path.abspath('../scripts'))

In [2]:

import nest_asyncio
from telethon.sync import TelegramClient, events
from telegram_scraper import create_telegram_client, scrape_telegram_messages  # Your scraper module

nest_asyncio.apply()

# Define your API credentials
api_id = '27914233'  # Replace with your API ID
api_hash = '57b3c112ce2bf9b13ec5a2190bff350a'  # Replace with your API Hash
phone = '+251966405464'  # Your phone number registered with Telegram
channel_username = 'ZemenExpress'  # Your Telegram channel username

# Initialize the Telegram client
client = TelegramClient('session_name', api_id, api_hash)

# Function to save real-time messages (text, images, documents)
async def save_real_time_message(message):
    SAVE_DIR = 'telegram_data/'
    if not os.path.exists(SAVE_DIR):
        os.makedirs(SAVE_DIR)

    if message.text:
        print(f"Text message: {message.text}")
        with open(f'{SAVE_DIR}/messages.txt', 'a', encoding='utf-8') as f:
            f.write(f'{message.date}: {message.text}\n')

    if message.photo:
        photo_path = await message.download_media(file=f'{SAVE_DIR}/photos/')
        print(f"Photo saved: {photo_path}")
    
    if message.document:
        document_path = await message.download_media(file=f'{SAVE_DIR}/documents/')
        print(f"Document saved: {document_path}")

# Real-time message event listener
@client.on(events.NewMessage(chats=channel_username))
async def real_time_message_handler(event):
    print(f"New message received in {channel_username}:")
    await save_real_time_message(event.message)

# Function to scrape old messages and listen for real-time messages
async def run_scraper():
    # Initialize the Telegram client
    client = await create_telegram_client(api_id, api_hash, phone)
    
    # Scrape old messages
    df = await scrape_telegram_messages(client, channel_username, limit=100)
    df.to_csv(f'{channel_username}_old_messages.csv', index=False)
    print("Old messages scraped and saved!")
    
    # Start real-time message listening
    print(f"Listening for real-time messages in {channel_username}...")
    await client.run_until_disconnected()

# Run the async function to scrape old and real-time messages
await run_scraper()


Old messages scraped and saved!
Listening for real-time messages in ZemenExpress...
